In [1]:
import math
from flashtext import KeywordProcessor
from db_conn import get_connection
import datetime

In [2]:
def get_table_ids():
    _cur = conn.cursor()
    select_sql = """select id from article_tables order by id"""
    _cur.execute(select_sql)
    return _cur.fetchall()

def get_side_effects():
    _cur = conn.cursor()
    select_sql = """select distinct(lower(llt_name)) from meddra_llt_180717 WHERE exclude=0"""
    _cur.execute(select_sql)
    return _cur.fetchall()

def get_drugs():
    _cur = conn.cursor()
    select_sql = """select distinct(lower(cui1_str)) from dict_collapsed_final"""
    _cur.execute(select_sql)
    return _cur.fetchall()

def get_sentences_to_search(ids):
    _cur = conn.cursor()
    select_sql = """select id, sentence from article_table_sentences where table_id in %s"""

    _cur.execute(select_sql, (tuple(ids),))
    row_count = _cur.rowcount
    return _cur.fetchall()

def get_sentences_to_search_m(ids):
    _cur = conn.cursor()
    select_sql_m = """select id, sentence from article_table_sentences_m where table_id in %s"""

    _cur.execute(select_sql_m, (tuple(ids),))
    row_count = _cur.rowcount
    
    return _cur.fetchall()

def copy_into_table(col, rows):
    _cur = conn.cursor()
    if col =='drug':
        _cur.executemany(
            '''
                UPDATE article_table_sentences
                SET
                    drug = %(drug)s
                WHERE
                    id = %(id)s
            ''',
            tuple(rows)
        )
    else:
        _cur.executemany(
            '''
                UPDATE article_table_sentences
                SET
                    adverse_effect = %(adverse_effect)s
                WHERE
                    id = %(id)s
            ''',
            tuple(rows)
        )

    row_count = _cur.rowcount
    conn.commit()
    print(row_count)

def copy_into_table_m(col, rows):
    _cur = conn.cursor()
    if col=='drug':
        _cur.executemany(
            '''
                UPDATE article_table_sentences_m
                SET
                    drug = %(drug)s
                WHERE
                    id = %(id)s
            ''',
            tuple(rows)
        )
    else:
                _cur.executemany(
            '''
                UPDATE article_table_sentences_m
                SET
                    adverse_effect = %(adverse_effect)s
                WHERE
                    id = %(id)s
            ''',
            tuple(rows)
        )

    row_count = _cur.rowcount
    conn.commit()
    print(row_count)

In [3]:
start_time = datetime.datetime.now()

conn = get_connection()

ids = list(map(lambda x:x[0], get_table_ids()))

def update_drugs(ids):
    drugs = list(map(lambda x: x[0], get_drugs()))
    keyword_processor = KeywordProcessor(case_sensitive=False)
    for d in drugs:
        keyword_processor.add_keyword(d.strip().encode('utf-8'))
        keyword_processor.add_keyword((' '+d.strip()+' ').encode('utf-8'))
        keyword_processor.add_keyword(('-'+d.strip()).encode('utf-8'))
        keyword_processor.add_keyword((d.strip()+'-').encode('utf-8'))
        
    update_list = []
    sentences = get_sentences_to_search(ids)
    for sen in sentences:
        id = sen[0]

        ori_found = []
        found = []
        found = keyword_processor.extract_keywords(sen[1].encode('utf-8').strip().lower())
        if len(found)>0:
            _found = list(set(list(map(lambda x: x.decode('utf-8'), found))))
            sen_strs = sen[1].encode('utf-8').strip().lower().split(' ')
            for f in _found:
                ori_found.extend(list(filter(lambda x: x.find(f)>=0, sen_strs)))
            print(sen[1].encode('utf-8').strip().lower())
            update_list.append({'id':id, 'drug':' '+' , '.join(list(set(ori_found)))+' '})

    if len(update_list)>0:
        print('update')
        print(update_list)
        copy_into_table('drug', update_list)
    
    update_list_m = []
    sentences_m = get_sentences_to_search_m(ids)
    for sen in sentences_m:
        id = sen[0]

        found = []
        found = keyword_processor.extract_keywords(sen[1].encode('utf-8').strip().lower())
        if len(found)>0:
            _found = list(map(lambda x: x.decode('utf-8'), found))
            print(_found)
            print(sen[1].encode('utf-8').strip().lower())
            update_list_m.append({'id':id, 'drug':' '+' , '.join(_found)+' '})

    if len(update_list_m)>0:
        print('update m')
        copy_into_table_m('drug', update_list_m)
        
def update_llts(ids):
    side_effects = list(map(lambda x: x[0], get_side_effects()))
    keyword_processor = KeywordProcessor(case_sensitive=False)
    for side in side_effects:
        keyword_processor.add_keyword((' '+side.strip()+' ').encode('utf-8'))
        keyword_processor.add_keyword(('-'+side.strip()).encode('utf-8'))
        keyword_processor.add_keyword((side.strip()+'-').encode('utf-8'))
    
    update_list = []
    sentences = get_sentences_to_search(ids)
    for sen in sentences:
        id = sen[0]

        found = []
        found = keyword_processor.extract_keywords(sen[1].encode('utf-8').strip().lower())
        if len(found)>0:
            _found = list(map(lambda x: x.decode('utf-8'), found))
            print(_found)
            print(sen[1].encode('utf-8').strip().lower())
            update_list.append({'id':id, 'adverse_effect':' '+' , '.join(_found)+' '})

    if len(update_list)>0:
        print('update')
        print(update_list)
        copy_into_table('ad', update_list)
    
    update_list_m = []
    sentences_m = get_sentences_to_search_m(ids)
    for sen in sentences_m:
        id = sen[0]

        found = []
        found = keyword_processor.extract_keywords(sen[1].encode('utf-8').strip().lower())
        if len(found)>0:
            _found = list(map(lambda x: x.decode('utf-8'), found))
            print(_found)
            print(sen[1].encode('utf-8').strip().lower())
            update_list_m.append({'id':id, 'adverse_effect':' '+' , '.join(_found)+' '})

    if len(update_list_m)>0:
        print('update m')
        copy_into_table_m('ad', update_list_m)

for i in range(1000, len(ids)+1000, 1000):
    print(str(i-1000)+':'+str(i))
    print('llts : ' +str(update_llts(ids[i-1000:i])))
    print('drugs:' + str(update_drugs(ids[i-1000:i])))
    
print(start_time - datetime.datetime.now())

0:1000
[' death ']
b'time from symptom onset to death , yr : 3'
[' death ']
b'age at death , yr : 39'
[' death ']
b'age at death , yr : 58'
['arrhythmia-']
b'follow-up: success (arrhythmia-free) : yes'
[' heart failure ']
b'congestive heart failure , 42%'
[' regurgitation ']
b'mitral regurgitation (baseline) : 1+'
[' shock ']
b'cardiogenic shock , odds ratio : 1.42'
[' stenosis ']
b'% diameter stenosis : 21.75'
[' stenosis ']
b'% diameter stenosis : 6.48'
[' tachycardia ']
b'ventricular tachycardia no. : 1'
[' pressure ']
b'right atrial pressure (mm hg) , p value : 0.47'
[' atrial fibrillation ']
b'svo2 (%) , atrial fibrillation : 64 \xc2\xb1 6'
[' tachycardia ']
b'ventricular tachycardia no. : 11'
[' stenosis ']
b'% diameter stenosis : 5.80'
[' occlusion ']
b'abrupt occlusion : yes'
[' stenosis ']
b'% diameter stenosis : 7.87'
[' stenosis ']
b'% diameter stenosis : 20.00'
[' stenosis ']
b'% diameter stenosis : 31.95'
[' stenosis ']
b'% diameter stenosis : 43.33'
[' stenosis ']
b'% dia

[' death ', ' myeloid leukaemia ', ' cancer ']
b'causes of death in men aged 35\xe2\x80\x9374 years , 10 myeloid leukaemia , urinary cancer , white smoking-attributed / total deaths (%) smoking-attributed fraction (% , from smoking-attributed number / total number) . : 13 / 377 (3\xc2\xb74%)'
[' death ', ' myeloid leukaemia ', ' cancer ']
b'causes of death in men aged 35\xe2\x80\x9374 years , 10 myeloid leukaemia , urinary cancer , african smoke (% of dead) : 45\xc2\xb75%'
[' ischaemic stroke ']
b'stroke , ischaemic stroke or unknown , lower-target group (n=1501) number of patients : 112'
[' death ', ' myeloid leukaemia ', ' cancer ']
b'causes of death in men aged 35\xe2\x80\x9374 years , 10 myeloid leukaemia , urinary cancer , african smoking-attributed / total deaths (%) smoking-attributed fraction (% , from smoking-attributed number / total number) . : 14 / 299 (4\xc2\xb77%)'
[' death ', ' myeloid leukaemia ', ' cancer ']
b'causes of death in men aged 35\xe2\x80\x9374 years , 10 mye

[' pressure ']
b'moderate to vigorous activity (min per day) , diet and activity vs usual care , between-group differences at 12 months (95% ci) adjusted for male or female sex , route into study (general practice , education centre , or responding to an advertisement) , age (continuous) , fitness (low [if maximum oxygen consumption poor or very poor on normative data] or high [if maximum oxygen consumption fair , good , excellent , or superior on normative data]) , study centre (taunton , north bristol , south bristol , or cheltenham and gloucester) , baseline glycated haemoglobin a1c (continuous) , and blood pressure (<120 / 80 mm\xe2\x80\x88hg , 120 / 80\xe2\x80\x93139 / 89 mm\xe2\x80\x88hg , or \xe2\x89\xa5140 / 90 mm\xe2\x80\x88hg) . : 5\xc2\xb762 (0\xc2\xb736 to 10\xc2\xb787)'
[' pressure ']
b'log fasting glucose (mmol / l) , diet and activity vs diet , between-group differences at 6 months (95% ci) adjusted for male or female sex , route into study (general practice , education 

b'crude culture filtrate proteins of mycobacterium tuberculosis ccfp cytokine responses were available for 1506 infants given bcg at entebbe hospital ; four missing values in this analysis for infants of women with no stool result for kato-katz analysis. , interleukin 13 , praziquantel effect of treatment in women without schistosomiasis (geometric mean ratio , 95% ci) : 1\xc2\xb704 (0\xc2\xb782\xe2\x80\x931\xc2\xb732)'
[' tuberculosis ', ' interaction ']
b'crude culture filtrate proteins of mycobacterium tuberculosis ccfp cytokine responses were available for 1506 infants given bcg at entebbe hospital ; four missing values in this analysis for infants of women with no stool result for kato-katz analysis. , interleukin 13 , praziquantel p value for interaction : 0\xc2\xb792'
[' tuberculosis ', ' hookworm infection ']
b'crude culture filtrate proteins of mycobacterium tuberculosis ccfp cytokine responses were available for 1506 infants given bcg at entebbe hospital ; four missing values

b'previous myocardial infarction (%) , 6% , 401\xe2\x80\x931,000 : 10%'
[' myocardial infarction ']
b'previous myocardial infarction (%) , 6% , >1,000 : 30%'
[' myocardial infarction ']
b'previous myocardial infarction (%) , 6% , p value : <0.0001'
[' coronary artery disease ']
b'known coronary artery disease (%) , 3% , 1\xe2\x80\x93400 : 17%'
[' coronary artery disease ']
b'known coronary artery disease (%) , 3% , 401\xe2\x80\x931,000 : 26%'
[' coronary artery disease ']
b'known coronary artery disease (%) , 3% , >1,000 : 60%'
[' coronary artery disease ']
b'known coronary artery disease (%) , 3% , p value : <0.0001'
[' angina ']
b'stable angina , coronary artery bypass graft surgery complete (n = 477) : 58.7%'
[' angina ']
b'stable angina , coronary artery bypass graft surgery incomplete (n = 90) : 58.9%'
[' angina ']
b'stable angina , percutaneous transluminal coronary angioplasty / stenting complete (n = 406) : 57.1%'
[' angina ']
b'stable angina , percutaneous transluminal coronar

...

update
[{'id': 6859, 'adverse_effect': '  death  '}, {'id': 6868, 'adverse_effect': '  death  '}, {'id': 6885, 'adverse_effect': '  death  '}, {'id': 7048, 'adverse_effect': ' arrhythmia- '}, {'id': 8242, 'adverse_effect': '  heart failure  '}, {'id': 19693, 'adverse_effect': '  regurgitation  '}, {'id': 52283, 'adverse_effect': '  shock  '}, {'id': 19234, 'adverse_effect': '  stenosis  '}, {'id': 19238, 'adverse_effect': '  stenosis  '}, {'id': 20004, 'adverse_effect': '  tachycardia  '}, {'id': 19769, 'adverse_effect': '  pressure  '}, {'id': 19771, 'adverse_effect': '  atrial fibrillation  '}, {'id': 20041, 'adverse_effect': '  tachycardia  '}, {'id': 19206, 'adverse_effect': '  stenosis  '}, {'id': 19207, 'adverse_effect': '  occlusion  '}, {'id': 19211, 'adverse_effect': '  stenosis  '}, {'id': 19215, 'adverse_effect': '  stenosis  '}, {'id': 19221, 'adverse_effect': '  stenosis  '}, {'id': 19226, 'adverse_effect': '  stenosis  '}, {'id': 19231, 'adverse_effect': '  stenosis  '}, 

14264
[' hypertrophic cardiomyopathy ', ' hypertrophic cardiomyopathy ', ' hypertrophic cardiomyopathy ', ' hypertrophic cardiomyopathy ']
b'protrusion height , cm , obstructive hypertrophic cardiomyopathy (n= 22) : 2.57 \xc2\xb1 0.3 p value for obstructive hypertrophic cardiomyopathy versus normal . p value for obstructive hypertrophic cardiomyopathy versus nonobstructive hypertrophic cardiomyopathy . p< 0.001 p< 0.001'
[' hypertrophic cardiomyopathy ']
b'protrusion height , cm , nonobhcm (n= 23) : 1.85 \xc2\xb1 0.6 pvalue for nonobstructive hypertrophic cardiomyopathy versus normal . p< 0.001'
[' hypertrophic cardiomyopathy ', ' hypertrophic cardiomyopathy ', ' hypertrophic cardiomyopathy ', ' hypertrophic cardiomyopathy ']
b'residual leaf length , cm , obstructive hypertrophic cardiomyopathy (n= 22) : 1.03 \xc2\xb1 0.3 p value for obstructive hypertrophic cardiomyopathy versus normal . p value for obstructive hypertrophic cardiomyopathy versus nonobstructive hypertrophic cardiomyopa

[' syncope ', ' ventricular tachycardia ']
b'pt no. ,  10 , syncope with ventricular tachycardia : presyncope'
[' dyspnea ']
b'variables ,  dyspnea , negative predictivevalue (%) : 61'
[' dyspnea ']
b'variables ,  dyspnea , accuracy (%) : 53'
[' dyspnea ']
b'variables ,  sensitivity (%) : dyspnea on exertion , 85'
[' dyspnea ']
b'variables ,  dyspnea on exertion , specificity (%) : 32'
[' dyspnea ']
b'variables ,  dyspnea on exertion , positive predictivevalue (%) : 48'
[' dyspnea ']
b'variables ,  dyspnea on exertion , negative predictivevalue (%) : 75'
[' dyspnea ']
b'variables ,  dyspnea on exertion , accuracy (%) : 54'
[' orthopnea ']
b'variables ,  sensitivity (%) : orthopnea , 46'
[' orthopnea ']
b'variables ,  orthopnea , specificity (%) : 83'
[' orthopnea ']
b'variables ,  orthopnea , positive predictivevalue (%) : 66'
[' orthopnea ']
b'variables ,  orthopnea , negative predictivevalue (%) : 68'
[' orthopnea ']
b'variables ,  orthopnea , accuracy (%) : 67'
[' depression ']
b'va

b'causes of death in men aged 35\xe2\x80\x9374 years , 10 myeloid leukaemia , urinary cancer , african smoking-attributed / total deaths (%) smoking-attributed fraction (% , from smoking-attributed number / total number) . : 14 / 299 (4\xc2\xb77%)'
[' death ', ' myeloid leukaemia ', ' cancer ']
b'causes of death in men aged 35\xe2\x80\x9374 years , 10 myeloid leukaemia , urinary cancer , other smoke (% of dead) : 50\xc2\xb78%'
[' death ', ' myeloid leukaemia ', ' cancer ']
b'causes of death in men aged 35\xe2\x80\x9374 years , 10 myeloid leukaemia , urinary cancer , other smoking-attributed / total deaths (%) smoking-attributed fraction (% , from smoking-attributed number / total number) . : 0 / 61'
[' death ', " parkinson's disease ", ' ulcerative colitis ']
b"causes of death in men aged 35\xe2\x80\x9374 years , 11 parkinson's disease , ulcerative colitis , coloured smoke (% of dead) : 41\xc2\xb72%"
[' death ', " parkinson's disease ", ' ulcerative colitis ']
b"causes of death in men 

KeyboardInterrupt: 